In [36]:
import pandas as pd
import requests
import os
import time
from bs4 import BeautifulSoup
from splinter import Browser

In [37]:
# Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text.
# https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest
url_nasa_news = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
response = requests.get(url_nasa_news)
nasa_soup = BeautifulSoup(response.text, 'html.parser')

In [38]:
nasa_news_title_raw = nasa_soup.find('div', class_='content_title').text
nasa_news_desc_raw = nasa_soup.find('div', class_='slide').find('div', class_='rollover_description').text

In [39]:
nasa_news_title = nasa_news_title_raw.replace('\n', '')
nasa_news_desc = nasa_news_desc_raw.replace('\n', '')

In [40]:
# scrape and save the featured image
# https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars
url_featured_img = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
response = requests.get(url_featured_img)
img_soup = BeautifulSoup(response.text, 'html.parser')

In [41]:
url_img_part_a = 'https://www.jpl.nasa.gov'
url_img_part_b = img_soup.find('div', class_='carousel_items').a['data-fancybox-href']
featured_img_url = url_img_part_a + url_img_part_b

In [42]:
raw_img_title = img_soup.find('div', class_='carousel_items').h1.text
img_title = raw_img_title.replace('\r','').replace('\n','').replace('\t','').strip()

In [43]:
# collect the most recent tweet from the Mars Weather twitter account
# https://twitter.com/marswxreport?lang=en
# use splinter
url_mars_weather = 'https://twitter.com/marswxreport?lang=en'

In [44]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [45]:
browser = init_browser()
browser.visit(url_mars_weather)
time.sleep(2)
weather_html = browser.html
weather_soup = BeautifulSoup(weather_html, "html.parser")
browser.quit()
print('Done with the browser')
mars_weather = weather_soup.body.div.main.article.find('div', lang='en').text

Done with the browser


In [46]:
# Collect and convert the html table from
# https://space-facts.com/mars/
url_mars_facts = 'https://space-facts.com/mars/'
response = requests.get(url_mars_facts)
facts_soup = BeautifulSoup(response.text, 'html.parser')

In [47]:
# pull the table
facts_table_list = facts_soup.table.find_all('td')

In [48]:
# each td in the list is on both sides
# description -> data -> description.
# they alternate back and forth
fact_text_list = []
for fact in facts_table_list:
    fact_text_list.append(fact.text)

In [49]:
# split apart the titels and data
x = 0
fact_title = []
fact_data = []
for text in fact_text_list:
    if (x%2 == 0):
        fact_title.append(text)
        x+=1
    else:
        fact_data.append(text)
        x+=1

In [50]:
# make a dict of the table using the titels as keys
# and the data as the value
dict_fact_table = {}
for i in range(len(fact_title)):
    dict_fact_table[fact_title[i]] = fact_data[i]

In [77]:
df = pd.DataFrame(dict_fact_table, index=[0])

In [79]:
df

,Equatorial Diameter:,Polar Diameter:,Mass:,Moons:,Orbit Distance:,Orbit Period:,Surface Temperature:,First Record:,Recorded By:
0,"6,792 km","6,752 km",6.39 × 10^23 kg (0.11 Earths),2 (Phobos & Deimos),"227,943,824 km (1.38 AU)",687 days (1.9 years),-87 to -5 °C,2nd millennium BC,Egyptian astronomers


In [80]:
df = df.T

In [83]:
df.columns = ['Value']

In [84]:
df

,Value
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [86]:
html_table = df.to_html()

In [52]:
# Obtain a hq img url for each of mars's 4 hemmispheres
# https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
url_mars_hemi = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
response = requests.get(url_mars_hemi)
hemi_soup = BeautifulSoup(response.text, 'html.parser')

In [53]:
hemi_a_tags = hemi_soup.body.find('div', class_ = 'container').find_all('a')

In [54]:
hemi_img_base_urls = []
hemi_img_titels = []
for tag in hemi_a_tags:
    hemi_img_base_urls.append(tag['href'])
    hemi_img_titels.append(tag.text)
hemi_img_base_urls.pop(0)
hemi_img_titels.pop(0)

'\n\nLunar and Planetary Cartographic Catalog\n'

In [55]:
hemi_final_urls = []
base_hemi_url = 'https://astrogeology.usgs.gov'
for target in hemi_img_base_urls:
    url = base_hemi_url + target
    response = requests.get(url)
    hemi_img_soup = BeautifulSoup(response.text, 'html.parser')
    hemi_final_urls.append(hemi_img_soup.body.find('div', class_ = 'container').find('div', class_ = 'downloads').a['href'])

In [56]:
hemisphere_image_urls = []
for x in range(len(hemi_img_titels)):
    dit = {}
    dit['title'] = hemi_img_titels[x]
    dit['img_url'] = hemi_final_urls[x]
    hemisphere_image_urls.append(dit)

In [58]:
hemisphere_image_urls[0]['img_url']

'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'

In [87]:
print(html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>
